# 문자 레벨 기계번역기
### 참조: [sequence-to-sequence 10분만에 이해하기](https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html)
### 다운로드: [프랑스-영어 병렬 코퍼스](http://www.manythings.org/anki)

## 1. 데이터 확인 및 전처리

In [36]:
import pandas as pd
lines = pd.read_csv('data/fra.txt', names=['src', 'dst', 'desc'], sep='\t')
len(lines)

170651

In [37]:
# 60,000개의 샘플만 가지고 기계 번역기를 구축
lines = lines[0:60000] # 6만개만 저장
lines.sample(5)

,src,dst,desc
22585,Everything changes.,Tout change.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
43697,We must do it quickly.,Nous devons le faire vite.,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
31581,We should do better.,Nous devrions faire mieux.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
11363,Is Tom an actor?,Tom est-il acteur ?,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
34175,I don't want to stop.,Je ne veux pas m'arrêter.,CC-BY 2.0 (France) Attribution: tatoeba.org #1...


In [38]:
# desc 컬럼 삭제
del lines['desc']

In [39]:
# dst 열에 <sos>로 \t, <eos>로 \n 을 추가
lines.dst = lines.dst.apply(lambda x : '\t '+ x + ' \n')
lines.sample(5)

,src,dst
18674,I heard something.,\t J'ai entendu quelque chose. \n
41111,I want you to kiss me.,\t Je veux que tu m'embrasses. \n
21359,What is happiness?,\t Qu'est-ce que le bonheur ? \n
45465,Don't let it scare you.,\t Ne vous laissez pas impressionner. \n
20853,Tom is handcuffed.,\t Tom est menotté. \n


In [11]:
# 글자 집합 생성 (단어가 아님)
src_vocab = set()
for line in lines.src: # 1줄씩 읽음
    for char in line: # 1개의 글자씩 읽음
        src_vocab.add(char)

dst_vocab = set()
for line in lines.dst:
    for char in line:
        dst_vocab.add(char)

In [12]:
src_vocab_size = len(src_vocab)+1
dst_vocab_size = len(dst_vocab)+1
print(src_vocab_size, dst_vocab_size)

79 106


In [13]:
src_vocab = sorted(list(src_vocab))
dst_vocab = sorted(list(dst_vocab))
print(src_vocab[45:75])
print(dst_vocab[45:75])

['W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
['T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w']


In [14]:
# 각 글자에 인덱스 부여
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
dst_to_index = dict([(word, i+1) for i, word in enumerate(dst_vocab)])
print(src_to_index)
print(dst_to_index)

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33, 'K': 34, 'L': 35, 'M': 36, 'N': 37, 'O': 38, 'P': 39, 'Q': 40, 'R': 41, 'S': 42, 'T': 43, 'U': 44, 'V': 45, 'W': 46, 'X': 47, 'Y': 48, 'Z': 49, 'a': 50, 'b': 51, 'c': 52, 'd': 53, 'e': 54, 'f': 55, 'g': 56, 'h': 57, 'i': 58, 'j': 59, 'k': 60, 'l': 61, 'm': 62, 'n': 63, 'o': 64, 'p': 65, 'q': 66, 'r': 67, 's': 68, 't': 69, 'u': 70, 'v': 71, 'w': 72, 'x': 73, 'y': 74, 'z': 75, 'é': 76, '’': 77, '€': 78}
{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '$': 6, '%': 7, '&': 8, "'": 9, '(': 10, ')': 11, ',': 12, '-': 13, '.': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, '?': 26, 'A': 27, 'B': 28, 'C': 29, 'D': 30, 'E': 31, 'F': 32, 'G': 33, 'H': 34, 'I': 3

In [15]:
# 영어 데이터에 대한 정수 인코딩
encoder_input = []
for line in lines.src: #입력 데이터에서 1줄씩 문장을 읽음
    temp_X = []
    for w in line: #각 줄에서 1개씩 글자를 읽음
      temp_X.append(src_to_index[w]) # 글자를 해당되는 정수로 변환
    encoder_input.append(temp_X)
print(encoder_input[:5])

[[30, 64, 10], [31, 58, 10], [31, 58, 10], [41, 70, 63, 2], [41, 70, 63, 2]]


In [16]:
# 프랑스어 데이터에 대한 정수 인코딩
decoder_input = []
for line in lines.dst:
    temp_X = []
    for w in line:
      temp_X.append(dst_to_index[w])
    decoder_input.append(temp_X)
print(decoder_input[:5])

[[1, 3, 48, 53, 3, 4, 3, 2], [1, 3, 45, 53, 64, 73, 72, 3, 4, 3, 2], [1, 3, 45, 53, 64, 73, 72, 14, 3, 2], [1, 3, 29, 67, 73, 70, 71, 105, 4, 3, 2], [1, 3, 29, 67, 73, 70, 57, 78, 105, 4, 3, 2]]


In [17]:
# 디코더의 예측값과 비교하기 위한 실제값
# 정수 인코딩 과정에서 <sos>를 제거
decoder_target = []
for line in lines.dst:
    t=0
    temp_X = []
    for w in line:
      if t>0:
        temp_X.append(dst_to_index[w])
      t=t+1
    decoder_target.append(temp_X)
print(decoder_target[:5])

[[3, 48, 53, 3, 4, 3, 2], [3, 45, 53, 64, 73, 72, 3, 4, 3, 2], [3, 45, 53, 64, 73, 72, 14, 3, 2], [3, 29, 67, 73, 70, 71, 105, 4, 3, 2], [3, 29, 67, 73, 70, 57, 78, 105, 4, 3, 2]]


In [18]:
# 패딩을 위해서 영어 문장과 프랑스어 문장 각각에 대해서 가장 길이가 긴 샘플의 길이 확인
max_src_len = max([len(line) for line in lines.src])
max_dst_len = max([len(line) for line in lines.dst])
print(max_src_len, max_dst_len)

25 76


In [19]:
# 평균 샘플 길이
print('영어 문장의 평균 길이 : {}'.format(sum(map(len, lines.src))/len(lines.src)))
print('불어 문장의 평균 길이 : {}'.format(sum(map(len, lines.dst))/len(lines.dst)))

영어 문장의 평균 길이 : 19.4515
불어 문장의 평균 길이 : 28.501116666666668


In [20]:
# 패딩시 사용할 크기 결정
pad_src_len = 25
pad_dst_len = 76

In [21]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
encoder_input = pad_sequences(encoder_input, maxlen=pad_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=pad_dst_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=pad_dst_len, padding='post')

In [22]:
# 원-핫 인코딩
from tensorflow.keras.utils import to_categorical
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

## 2. 교사 강요(Teacher forcing)
- 훈련 과정에서는 이전 시점의 디코더 셀의 출력을 현재 시점의 디코더 셀의 입력으로 넣어주지 않고,
- 이전 시점의 실제값을 현재 시점의 디코더 셀의 입력값으로 하는 방법을 사용
- RNN의 모든 시점에 대해서 이전 시점의 예측값 대신 실제값을 입력으로 주는 방법을 교사 강요라고 함

## 3. seq2seq 기계 번역기 훈련시키기

In [23]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model

In [24]:
encoder_inputs = Input(shape=(None, src_vocab_size), name='Encoder_Input')
encoder_lstm = LSTM(units=256, return_state=True, name='Encoder_LSTM')
# 인코더의 내부 상태를 디코더로 넘겨주어야 하기 때문에 return_state=True로 설정
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# encoder_outputs도 같이 리턴받기는 했지만 여기서는 필요없으므로 이 값은 버림.
encoder_states = [state_h, state_c]
# LSTM은 바닐라 RNN과는 달리 상태가 두 개. 바로 은닉 상태와 셀 상태.
# 이겻이 컨텍스트 벡터

In [25]:
decoder_inputs = Input(shape=(None, dst_vocab_size), name='Decoder_Input')
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True, name='Decoder_LSTM')
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)
# 디코더의 첫 상태를 인코더의 은닉 상태, 셀 상태로 합니다.

In [26]:
decoder_softmax_layer = Dense(dst_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Encoder_Input (InputLayer)      [(None, None, 79)]   0                                            
__________________________________________________________________________________________________
Decoder_Input (InputLayer)      [(None, None, 106)]  0                                            
__________________________________________________________________________________________________
Encoder_LSTM (LSTM)             [(None, 256), (None, 344064      Encoder_Input[0][0]              
__________________________________________________________________________________________________
Decoder_LSTM (LSTM)             [(None, None, 256),  371712      Decoder_Input[0][0]              
                                                                 Encoder_LSTM[0][1]           

In [27]:
from tensorflow.keras.utils import plot_model
plot_model(model, 'image/seq2seq.png')

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


In [28]:
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

In [29]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
modelpath = "model/seq2seq-{epoch:02d}-{val_loss:.4f}.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', 
                               verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

In [ ]:
model.fit(x=[encoder_input, decoder_input], y=decoder_target,
          batch_size=64, epochs=50, validation_split=0.2,
          callbacks=[checkpointer, early_stopping_callback])

Train on 48000 samples, validate on 12000 samples
Epoch 1/50
47936/48000 [============================>.] - ETA: 0s - loss: 0.7843
Epoch 00001: val_loss improved from inf to 0.69868, saving model to model/seq2seq-01-0.6987.hdf5
48000/48000 [==============================] - 249s 5ms/sample - loss: 0.7840 - val_loss: 0.6987
Epoch 2/50
47936/48000 [============================>.] - ETA: 0s - loss: 0.4848
Epoch 00002: val_loss improved from 0.69868 to 0.56263, saving model to model/seq2seq-02-0.5626.hdf5
48000/48000 [==============================] - 238s 5ms/sample - loss: 0.4847 - val_loss: 0.5626
Epoch 3/50
47936/48000 [============================>.] - ETA: 0s - loss: 0.4014
Epoch 00003: val_loss improved from 0.56263 to 0.49306, saving model to model/seq2seq-03-0.4931.hdf5
48000/48000 [==============================] - 242s 5ms/sample - loss: 0.4014 - val_loss: 0.4931
Epoch 4/50
 1280/48000 [..............................] - ETA: 3:49 - loss: 0.3741

In [30]:
# Best Model 선택
from tensorflow.keras.models import load_model
del model
model = load_model('model/seq2seq-03-0.4931.hdf5')

## 4. seq2seq 기계 번역기 동작시키기

In [31]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [32]:
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
# 문장의 다음 단어를 예측하기 위해서 초기 상태를 이전 상태로 사용
decoder_states = [state_h, state_c]
# 이번에는 훈련 과정에서와 달리 은닉 상태와 셀 상태인 state_h와 state_c를 버리지 않음.
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [33]:
index_to_src = dict(
    (i, char) for char, i in src_to_index.items())
index_to_dst = dict(
    (i, char) for char, i in dst_to_index.items())

In [34]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)
    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, dst_vocab_size))
    target_seq[0, 0, dst_to_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ""
    while not stop_condition: #stop_condition이 True가 될 때까지 루프 반복
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_dst[sampled_token_index]
        decoded_sentence += sampled_char

        # <sos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_dst_len):
            stop_condition = True

        # 길이가 1인 타겟 시퀀스를 업데이트 합니다.
        target_seq = np.zeros((1, 1, dst_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.

        # 상태를 업데이트 합니다.
        states_value = [h, c]

    return decoded_sentence

In [35]:
import numpy as np
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', lines.src[seq_index])
    print('정답 문장:', lines.dst[seq_index][1:len(lines.dst[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1]) # '\n'을 빼고 출력

-----------------------------------
입력 문장: Run!
정답 문장:  Cours ! 
번역기가 번역한 문장: zzz»z»zwèzwB« «YeekhTSgvISSSçISr&ekhTT?yyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy
-----------------------------------
입력 문장: I lost.
정답 문장:  J'ai perdu. 
번역기가 번역한 문장: zzz»z»zwèzwB« «YeekhTSgvISSSçISr&ekhTT?yyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy
-----------------------------------
입력 문장: Come in.
정답 문장:  Entre ! 
번역기가 번역한 문장: zzz»z»zwèzwB« «YeekhTSgvISSSçISr&ekhTT?yyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy
-----------------------------------
입력 문장: I got it.
정답 문장:  J'ai capté. 
번역기가 번역한 문장: zzz»z»zwèzwB« «YeekhTSgvISSSçISr&ekhTT?yyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy
-----------------------------------
입력 문장: Who cares?
정답 문장:  Qui s'en préoccupe ? 
번역기가 번역한 문장: zzz»z»zwèzwB« «YeekhTSgvISSSçISr&ekhTT?yyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy
